In [12]:
from mp_api.client import MPRester
from emmet.core.summary import HasProps
import json
from tqdm import tqdm
import os


my_api_key = "pdDir9sfmf4JPMKDSJzF8W7ZJ7CyqYHE"
file_path = "../mpids_downloaded.json"

with open(file_path, 'r') as file:
    mpids = json.load(file)

print("Len mpids:",len(mpids))

Len mpids: 117535


In [13]:
with open("mpids_to_lattice_type.json", 'r') as file:
    mpids_to_lattice_type = json.load(file)
with open("lattice_type_dict.json", 'r') as file:
    lattice_type_dict = json.load(file)

In [2]:
# num cubic data 14421/1000/1000

In [14]:
print("Len mpids_to_lattice_type:",len(mpids_to_lattice_type))

Len mpids_to_lattice_type: 117535


In [15]:
for keys in lattice_type_dict:
    print(keys, len(lattice_type_dict[keys]))

triclinic 14843
monoclinic 28081
orthorhombic 24115
tetragonal 14031
trigonal 10651
hexagonal 8681
cubic 17133
unknown 0


In [5]:
lattice_type_dict["cubic"]

['mp-971767',
 'mp-1181814',
 'mp-1182639',
 'mp-159',
 'mp-1226548',
 'mp-1080740',
 'mp-1173545',
 'mp-14921',
 'mp-556182',
 'mp-1212420',
 'mp-1220683',
 'mp-760770',
 'mp-28580',
 'mp-1177872',
 'mp-1218922',
 'mp-772493',
 'mp-1246998',
 'mp-561536',
 'mp-1101793',
 'mp-772563',
 'mp-1019606',
 'mp-1105852',
 'mp-691038',
 'mp-1182129',
 'mp-766698',
 'mp-22131',
 'mp-1220344',
 'mp-1047709',
 'mp-974322',
 'mp-1201489',
 'mp-972220',
 'mp-1205869',
 'mp-1096631',
 'mp-1019710',
 'mp-778756',
 'mp-1112341',
 'mp-1182305',
 'mp-1226491',
 'mp-560071',
 'mp-1215349',
 'mp-1045347',
 'mp-11754',
 'mp-1111211',
 'mp-1182029',
 'mp-1210187',
 'mp-754275',
 'mp-1203850',
 'mp-1247543',
 'mp-1185577',
 'mp-975383',
 'mp-769612',
 'mp-559977',
 'mp-1105411',
 'mp-771133',
 'mp-23390',
 'mp-1201564',
 'mp-772651',
 'mp-759990',
 'mp-1228458',
 'mp-30548',
 'mp-1191831',
 'mp-1191568',
 'mp-1200633',
 'mp-1221255',
 'mp-505332',
 'mp-1094677',
 'mp-1174315',
 'mp-27607',
 'mp-865534',
 'mp

In [16]:
import random
random.seed(1)
train_lattice = {}
val_lattice = {}
test_lattice = {}
split = {
    "triclinic": 1000,
    "monoclinic": 1000,
    "orthorhombic": 1000,
    "tetragonal": 1000,
    "trigonal": 500,
    "hexagonal": 500,
    "cubic": 1000
}


for keys in lattice_type_dict:
    if keys == "unknown":
        continue
    random.shuffle(lattice_type_dict[keys])
    split_lat = split[keys]
    test_lattice[keys] = lattice_type_dict[keys][:split_lat]
    val_lattice[keys] = lattice_type_dict[keys][split_lat:split_lat*2]
    train_lattice[keys] = lattice_type_dict[keys][split_lat*2:]
    print(keys, len(lattice_type_dict[keys]), len(train_lattice[keys]), len(val_lattice[keys]), len(test_lattice[keys]))

triclinic 14843 12843 1000 1000
monoclinic 28081 26081 1000 1000
orthorhombic 24115 22115 1000 1000
tetragonal 14031 12031 1000 1000
trigonal 10651 9651 500 500
hexagonal 8681 7681 500 500
cubic 17133 15133 1000 1000


In [18]:
for keys in lattice_type_dict:
    if keys == "unknown":
        continue
    save_dict = {"train": train_lattice[keys], "validation": val_lattice[keys], "test": test_lattice[keys]}
    with open("mp_splits_true/mp_"+keys+"_true.json", 'w') as file:
        json.dump(save_dict, file)

In [19]:
save_list = {}
for keys in lattice_type_dict:
    if keys == "unknown":
        continue
    save_list[keys] = train_lattice[keys] + val_lattice[keys] + test_lattice[keys]
    with open("mp_splits_true/mp_"+keys+"_download.txt", 'w') as file:
        for mp in save_list[keys]:
            file.write(mp+".chgcar"+"\n")

In [20]:
load_list = {}
for keys in lattice_type_dict:
    if keys == "unknown":
        continue
    with open("mp_splits_true/mp_"+keys+"_download.txt", 'r') as file:
        load_list[keys] = file.read().splitlines()
    with open("mp_splits_true/mp_"+keys+"_download_lz4.txt", 'w') as file:
        for mp in load_list[keys]:
            file.write(mp+".lz4"+"\n")

In [21]:
import random
random.seed(1)

train_all = []
val_all = []
test_all = []
# sampling train, val, test
n_train = 2000
n_val = 150
n_test = 150
for keys in lattice_type_dict:
    print(keys, len(lattice_type_dict[keys]))
    # Random sampling
    random.shuffle(lattice_type_dict[keys])
    train = lattice_type_dict[keys][:n_train]
    val = lattice_type_dict[keys][n_train:n_train+n_val]
    test = lattice_type_dict[keys][n_train+n_val:n_train+n_val+n_test]
    train_all += train
    val_all += val
    test_all += test

triclinic 14843
monoclinic 28081
orthorhombic 24115
tetragonal 14031
trigonal 10651
hexagonal 8681
cubic 17133
unknown 0


In [ ]:
# scp -r holywater3@10.2.140.1:~/hw_data1/dataset_mp/data/{"$(cat folder_list.txt | tr '\n' ',' | sed 's/,$//')"} .
# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_trigonal_download.txt ~/hw_data1/dataset_mp/data holywater2@10.2.140.9:~/crystal/dataset_mp
# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_hexagonal_download.txt ~/hw_data1/dataset_mp/data holywater2@10.2.140.10:~/crystal/dataset_mp

# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_orthorhombic_download.txt ~/hw_data1/dataset_mp/data holywater2@10.2.140.9:~/crystal/dataset_mp_mixed #1.56T
# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_monoclinic_download.txt ~/hw_data1/dataset_mp/data holywater2@10.2.140.10:~/crystal/dataset_mp_mixed #1.84T

# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_triclinic_download.txt ~/hw_data1/dataset_mp/data holywater2@10.2.140.9:~/crystal/dataset_mp #0.96T
# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_cubic_downlad.txt ~/hw_data1/dataset_mp/data holywater2@10.2.140.10:~/crystal/dataset_mp #1.13T

# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_tetragonal_download_lz4.txt ~/hw_data1/dataset_mp/data_lz4 holywater2@10.2.140.6:~/crystal/dataset_mp_lz4

# rsync -avuh --progress --partial-dir=/tmp --files-from=mp_tetragonal_download.txt ~/hw_data1/dataset_mp/data  holywater2@10.2.140.9:~/crystal/dataset_mp_mixed


#------

# rsync -e 'ssh -p 7777' -avuh --progress --partial-dir=/tmp --files-from=mp_triclinic_download.txt ~/hw_data1/dataset_mp/data seongsukim@141.223.25.21:~/crystal/dataset_mp_mixed
# rsync -e 'ssh -p 7777' -avuh --progress --partial-dir=/tmp --files-from=mp_cubic_download.txt ~/hw_data1/dataset_mp/data seongsukim@141.223.25.21:~/crystal/dataset_mp_mixed


In [29]:
print(lattice_type_dict["triclinic"].index("mp-771910"))
train_all[train_all.index("mp-771910")] = lattice_type_dict["triclinic"][n_train+n_val+n_test]

942


In [30]:
error_files = ["mp-1201606","mp-771910","mp-32018","mp-1209684"]

In [31]:
lattice_count = {
    "cubic": 0,
    "tetragonal": 0,
    "orthorhombic": 0,
    "hexagonal": 0,
    "monoclinic": 0,
    "triclinic": 0,
    "trigonal": 0,
}
new = []
for error_file in error_files:
    lattice_type = mpids_to_lattice_type[error_file]
    if train_all.count(error_file) == 1:
        train_all.remove(error_file)
        train_all.append(lattice_type_dict[lattice_type][n_train+n_val+n_test + lattice_count[lattice_type]])
        lattice_count[lattice_type] += 1
        new.append(error_file)
    elif val_all.count(error_file) == 1:
        val_all.remove(error_file)
        val_all.append(lattice_type_dict[lattice_type][n_train+n_val+n_test + lattice_count[lattice_type]])
        lattice_count[lattice_type] += 1
        new.append(error_file)
    elif test_all.count(error_file) == 1:
        test_all.remove(error_file)
        test_all.append(lattice_type_dict[lattice_type][n_train+n_val+n_test + lattice_count[lattice_type]])
        lattice_count[lattice_type] += 1
        new.append(error_file)

In [ ]:
# [!] density shape not match 3559 mp-1201606
# density:  2140890
# grid shape:  [108, 160, 196] 3386880
# [!] density shape not match 4936 mp-771910
# density:  1298
# grid shape:  [84, 84, 140] 987840
# [!] density shape not match 5818 mp-32018
# density:  1121875
# grid shape:  [160, 160, 80] 2048000
# [!] density shape not match 10433 mp-1209684
# density:  1095975
# grid shape:  [112, 112, 112] 1404928


In [27]:
len(val_all), len(test_all), len(train_all)

(1050, 1050, 14000)

In [34]:
import shutil
import os

source_folder = "../data"
destination_folder = "../mixed_data_fixed"

# Copy all files from source folder to destination folder
for filename in error_files:
    filename_ext = filename + ".chgcar"
    source_file = os.path.join(source_folder, filename_ext)
    destination_file = os.path.join(destination_folder, filename_ext)
    shutil.copy(source_file, destination_file)


In [26]:
import shutil
import os

source_folder = "../data"
for key in lattice_type_dict:
    destination_folder = "../data_"+key
    os.makedirs(destination_folder, exist_ok=True)
    
    # Copy all files from source folder to destination folder
    for filename in save_list[key]:
        filename_ext = filename + ".chgcar"
        source_file = os.path.join(source_folder, filename_ext)
        destination_file = os.path.join(destination_folder, filename_ext)
        shutil.copy(source_file, destination_file)

KeyboardInterrupt: 

In [ ]:
# scp -r holywater3@10.2.140.1:~/hw_data1/dataset_mp/mixed_data ./dataset_mp_mixed